In [ ]:
#packages install
import geopandas as gpd
import pandas as pd
import json

In [ ]:
#Load Prefecure layer
pref_path = "../data/cleaned/polbnda_jpn_dissolved_points.geojson"

pref_gdf = gpd.read_file(pref_path)

In [ ]:
#Create x, y column
pref_gdf['longitude'] = pref_gdf.geometry.x
pref_gdf['latitude'] = pref_gdf.geometry.y

In [ ]:
pref_gdf.head()

In [ ]:
pref_gdf_selected = pref_gdf[['Origin_id', 'longitude', 'latitude']]

In [ ]:
#Load OD matrix csv
flow_file = "../data/cleaned/MergedFlow_2014-2022.csv"

df = pd.read_csv(flow_file)

In [ ]:
df

In [ ]:
#Add column origX, OrigY
merged = pd.merge(df, pref_gdf_selected, on = 'Origin_id', how = 'inner')
merged.rename(columns={'latitude': 'origY', 'longitude': 'origX'}, inplace=True)

In [ ]:
merged.info()

In [ ]:
#destX, destY
merged2 = pd.merge(merged, pref_gdf_selected, left_on = 'Destination_id', right_on = 'Origin_id', how = 'inner')
merged2.rename(columns={'latitude': 'destY', 'longitude': 'destX'}, inplace=True)

In [ ]:
merged2

In [ ]:
merged2 = merged2.drop(columns=['Origin_id_y'])

In [ ]:
merged2.rename(columns={'origin_id': 'Origin_id'}, inplace=True)

In [ ]:
#Transform
output = []

for _, row in merged2.iterrows():
    transformed_row = {
        "origin_id": int(row["Origin_id_x"]),
        "destination_id": int(row["Destination_id"]),
        "origin_coordinates": [row["origX"], row["origY"]],
        "destination_coordinates": [row["destX"], row["destY"]],
        "flow2014": float(row["Flow2014"]),
        "flow2015": float(row["Flow2015"]),
        "flow2016": float(row["Flow2016"]),
        "flow2017": float(row["Flow2017"]),
        "flow2018": float(row["Flow2018"]),
        "flow2019": float(row["Flow2019"]),
        "flow2022": float(row["Flow2022"]),
                      }
    output.append(transformed_row)

#Convert the result to JSON
output_json = json.dumps(output, indent=2)

In [ ]:
#Create GeoJSON 
output_geojson = []

for _, row in merged2.iterrows():
    feature = {
        "type": "Feature",
        "geometry": {
            "type": "LineString",
            "coordinates": [
                [row["origX"], row["origY"]],
                [row["destX"], row["destY"]]
            ]
        },
        "properties": {
            "origin_id": int(row["Origin_id"]),
            "destination_id": int(row["Destination_id"]),
            "flow2014": float(row["Flow2014"]),
            "flow2015": float(row["Flow2015"]),
            "flow2016": float(row["Flow2016"]),
            "flow2017": float(row["Flow2017"]),
            "flow2018": float(row["Flow2018"]),
            "flow2019": float(row["Flow2019"]),
            "flow2022": float(row["Flow2022"])
        }
    }
    output_geojson.append(feature)

#GeoJSON FeatureCollection
geojson_output = {
    "type": "FeatureCollection",
    "features": output_geojson
}

#Convert the result to GeoJSON
output_geojson_str = json.dumps(geojson_output, indent=2)

print(output_geojson_str)

In [ ]:
with open("../data/cleaned/od-flow-2014-2022.geojson", "w") as f:
    f.write(output_json)

In [ ]:
with open("../data/cleaned/od-flow-2014-2022.geojson", "w") as f:
    f.write(output_geojson_str)